In [ ]:
! git clone https://github.com/hemulitch/programming_3rd_year.git

In [ ]:
! pip install pymorphy2

#### Импорты

In [3]:
import pandas as pd
import nltk
nltk.download("stopwords")
nltk.download('punkt')
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#### Данные

In [4]:
dataset = pd.read_csv('/content/programming_3rd_year/hw1/result.csv', delimiter=';')

In [5]:
dataset = dataset.drop(['Unnamed: 0'], axis=1)
dataset.head()

,Отзыв,Оценка
0,\n \n\n\n\n\n\n\n\n\n\n\nЯ хотела бы поделитьс...,5.0
1,Давно искала идеальный тональный крем. Фирму K...,5.0
2,Доброе утро!☀️ Недавно купила тональный крем G...,3.0
3,Всем привет. Продолжая поиски идеального тонал...,3.0
4,О данном тональном креме я узнала пару месяцев...,5.0


In [6]:
# удаляем пустые строчки
dataset = dataset.dropna()

#### Метки классов

Отзывы с оценками 1 и 2 мы будем классифицировать как негативные, а с оценками 4 и 5 - как позитивные.

In [7]:
dataset['Оценка'].value_counts()

5.0    227
4.0    123
3.0     52
2.0     20
1.0     15
Name: Оценка, dtype: int64

Однако, распределение классов в полученном датасете очень неравномерное, поэтому я решила вручную выбрать отзывы, с оценкой 3, которые больше похожи на позитивные, чем на негативные, и убрать их из таблицы, а оставшиеся отзывы с оценкой 3 добавить в класс негативных отзывов

In [8]:
dataset = dataset.drop([3, 9, 27, 34, 38, 48, 54, 83, 88, 95, 108, 130, 135, 144, 188, 234, 267, 272, 278, 282, 303, 346, 371, 375, 393,
                        405, 416, 417])

Теперь переведем отзывы в два класса: положительные отзывы - 1, негативные отзывы - 0

In [9]:
grades_to_binary = [1 if grade in [4, 5] else 0 for grade in list(dataset['Оценка'])]
dataset['Класс'] = grades_to_binary

In [10]:
# разделим на 2 набора данных
data_neg = dataset[dataset['Класс'] == 0]
data_pos = dataset[dataset['Класс'] == 1]

In [11]:
# уменьшаем размер data_pos, чтобы не было дисбаланса классов
data_pos = data_pos[:59]

In [12]:
# отдельно выделим отзывы для теста, отдельно для "обучения"
train_neg = data_neg[:50]
train_pos = data_pos[:50]
test_neg = data_neg[50:]
test_pos = data_pos[50:]

In [13]:
print(f'TRAIN:\n positive reviews - {len(train_pos)}; negative reviews - {len(train_neg)}')
print('\n')
print(f'TEST:\n positive reviews - {len(test_pos)}; negative reviews - {len(test_neg)}')

TRAIN:
 positive reviews - 50; negative reviews - 50


TEST:
 positive reviews - 9; negative reviews - 9


#### Обработка текста

In [26]:
def preprocessing(text):

  # токенизация
  tokens = nltk.word_tokenize(text)

  # убираем знаки препинания, всё что не слова + приводим к нижнему регистру
  tokens = [w.lower() for w in tokens if w.isalpha()]

  # убираем все стоп-слова
  russian_stopwords = stopwords.words("russian")
  tokens = [w for w in tokens if w not in russian_stopwords]

  # убираем слова, написанные латиницей (поскольку это часто названия, они не имеют значения для классификации)
  new_tokens = []
  for word in tokens:
    counter = 0
    for char in word:
      if char in 'qwertyuiopasdfghjklzxcvbnm':
        counter += 1
    if counter == 0:
      new_tokens.append(word)

  preprocessed_text = " ".join(new_tokens)
  return preprocessed_text

In [ ]:
train_neg['Текст'] = train_neg['Отзыв'].apply(preprocessing)
train_pos['Текст'] = train_pos['Отзыв'].apply(preprocessing)

In [16]:
def lemmatize(text):

  tokens = text.split(' ')

  # начальная форма
  morph = MorphAnalyzer()
  for i, w in enumerate(tokens):
    ana = morph.parse(w)
    first = ana[0].normal_form  # лемма
    tokens[i] = first

  lemmatized_text = " ".join(tokens)

  return lemmatized_text

In [ ]:
train_neg['Леммы'] = train_neg['Текст'].apply(lemmatize)
train_pos['Леммы'] = train_pos['Текст'].apply(lemmatize)

#### Частотные слова в положительных и отрицательных отзывах

In [18]:
# функция для подсчета самых частых слов в отзывах
def frequency_counter(reviews, max_len):
    frequencies = Counter()
    for review in reviews:
      words = nltk.word_tokenize(review.lower())
      for word in words:
        if word.isalpha():
          frequencies[word] += 1
    return dict(frequencies.most_common(max_len))

In [57]:
neg_freqlist = frequency_counter(train_neg['Леммы'], 500)
pos_freqlist = frequency_counter(train_pos['Леммы'], 500)

In [58]:
# находим уникальные слова в положительных и отрицательных отзывах
neg_set = set(neg_freqlist.keys())
pos_set = set(pos_freqlist.keys())
neg_unique = neg_set - pos_set
pos_unique = pos_set - neg_set

In [59]:
# составляем словарь, в котором ключи - тональность отзыва, а значения - список самых частых слов в отзывах этого класса
frequency_dict = {}
frequency_dict['positive'] = pos_unique
frequency_dict['negative'] = neg_unique

#### Результаты

In [22]:
# обрабатываем отзывы
test = pd.concat([test_neg, test_pos])
test['Текст'] = test['Отзыв'].apply(preprocessing)
test['Леммы'] = test['Текст'].apply(lemmatize)

In [23]:
# функция для определения тональности отзыва
def sentiment_detector(freqdict, review):
    counts = Counter()
    # подсчитываем, как много слов в отзыве встречаются в neg_unique либо в pos_unique
    for sentiment, unique_words in freqdict.items():
        unique_words = Counter(unique_words)
        for word in nltk.word_tokenize(review):
          if unique_words[word] > 0:
            counts[sentiment] += 1
    return counts.most_common()

In [60]:
predictions = []
for review in test['Леммы']:
  predictions.append(sentiment_detector(frequency_dict, review))

In [61]:
predicted_y = [pred[0][0] for pred in predictions]
predicted_y = [0 if y == 'negative' else 1 for y in predicted_y]
true_y = test['Класс'].tolist()

print("Accuracy: %.4f" % accuracy_score(predicted_y, true_y))

Accuracy: 0.8889


Я поэкспереминтировала с количеством наиболее распространенных слов, которое выдает функция frequency_counter. Качество работы данного алгоритма зависит от этого параметра, ср.:

- n = 1000, accuracy = 0.5556
- n = 500 или n = 400, accuracy = 0.8889 - лучший результат
- n = 300, accuracy = 0.7222
- n = 200, accuracy = 0.8333
- n = 100, accuracy = 0.6667
- n = None, accuracy = 0.6667

#### Как можно улучшить этот результат?

- Считать не слова, а n-граммы, а так же не убирать стоп-слова. В этих отзывах часто встречаются похожие выражения и в отрицательных, и в положительных отзывах, например, "отдает рыжиной"/"не отдает рыжиной" или "подчеркивает высыпания"/"не подчеркивает высыпания". При подсчете частотности отдельных слов эта информация упускается;

- Векторизовать текст с помощью, например, Tf-idf и обучить логистическую регрессию для дальнейшей классификации тестовых данных